Objective: Demonstrate how to frame a simple optimization problem: choosing the cheapest shipping mode that meets a delivery deadline.

Frame the Problem
Let's find the average cost and shipping time for each shipping_mode from our data.

In [1]:
import pandas as pd
import pulp

df = pd.read_csv('../data/processed/final_features.csv')

# Calculate average cost (negative benefit) and time for each mode
# We use 'benefit_per_order' * -1 as a proxy for cost
shipping_options = df.groupby('shipping_mode').agg(
    avg_cost=('benefit_per_order', lambda x: x.mean() * -1),
    avg_time=('days_for_shipping_real', 'mean')
).reset_index()

print("Shipping Options (Averages):")
print(shipping_options)

Shipping Options (Averages):
    shipping_mode   avg_cost  avg_time
0     First Class -23.122238  2.000000
1        Same Day -20.850203  0.478279
2    Second Class -21.305889  3.990828
3  Standard Class -21.999169  3.995907


Define and Solve the Optimization Problem
Scenario: A customer needs an order delivered within 3 days. Which shipping mode should we choose to minimize cost?

In [2]:
# 1. Initialize the problem
prob = pulp.LpProblem("Shipping_Mode_Optimization", pulp.LpMinimize)

# 2. Define Decision Variables
# Create a binary variable for each shipping mode (0=Don't use, 1=Use)
modes = shipping_options['shipping_mode']
variables = pulp.LpVariable.dicts("Mode", modes, cat='Binary')

# 3. Define the Objective Function (minimize total cost)
prob += pulp.lpSum([shipping_options.set_index('shipping_mode').loc[i, 'avg_cost'] * variables[i] for i in modes])

# 4. Define Constraints
# Constraint 1: The chosen mode's time must be <= 3 days
prob += pulp.lpSum([shipping_options.set_index('shipping_mode').loc[i, 'avg_time'] * variables[i] for i in modes]) <= 3
# Constraint 2: We must choose exactly one shipping mode
prob += pulp.lpSum([variables[i] for i in modes]) == 1

# 5. Solve the problem
prob.solve()

# Print the results
print("\n--- Optimization Result ---")
print(f"Status: {pulp.LpStatus[prob.status]}")
for v in prob.variables():
    if v.varValue > 0:
        print(f"Optimal Choice: {v.name.replace('Mode_', '')}")
print(f"Minimum Cost (Proxy): ${pulp.value(prob.objective):.2f}")


--- Optimization Result ---
Status: Optimal
Optimal Choice: First_Class
Minimum Cost (Proxy): $-23.12


This simple example introduces you to the powerful world of prescriptive analytics, where you can make optimal decisions based on data and constraints.